# Customer-specialist
Agent responsible of solve fix to customer using requirementes describe on KB procedures

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://raw.githubusercontent.com/Foiac/customerSpecialist/refs/heads/main/customer-specialist/FlowDrawn.png" alt="Clean Data" style="width: 800px">
</div>

In [1]:
!pip install azure-search-documents

     -------------------------------------- 298.8/298.8 kB 9.3 MB/s eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Paths

In [26]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from openai import AzureOpenAI
import os
from dotenv import load_dotenv

### Confs

In [ ]:
load_dotenv()

AZURE_ENDPOINT = os.getenv("AZURE_OPEN_AI_ENDPOINT")

endpoint = f"https://{SERVICE_NAME}.search.windows.net"

service_name = "ai-search-procedures"
index_name = "procedimentos-kb"
api_key = "Yn3qJZOcOZZ95wMyIVdezvtwrtQ0OV32eg1yWKmAzJAzSeB88LR4"
endpoint = f"https://{service_name}.search.windows.net"

### Index Client

In [10]:
search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=AzureKeyCredential(api_key))

### OpenAI Client

In [ ]:
client = AzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint=AZURE_ENDPOINT,
    api_key="90Q0ww6bZpijSATwFOM0SW990z8V9a2b5VN6fNpBPU6dOChRwMKdJQQJ99BFACHYHv6XJ3w3AAAAACOGQIxp",
)

### Context function

In [14]:
def obter_contexto_rag(pergunta):
    resultados = search_client.search(pergunta, top=3)
    return "\n".join([doc['content'] for doc in resultados])

### Tools

In [15]:
tools = [{
    "type": "function",
    "function": {
        "name": "resetar_senha",
        "description": "Redefine a senha de um colaborador autorizado",
        "parameters": {
            "type": "object",
            "properties": {
                "cpf": {"type": "string"},
                "tipo_usuario": {"type": "string", "enum": ["colaborador", "gerente", "rh"]}
            },
            "required": ["cpf", "tipo_usuario"]
        }
    }
}]

### Define agent function

In [24]:
def chamar_agente(pergunta, contexto):
    resposta = client.chat.completions.create(
        model="gpt-4o-mini", 
        temperature=0,
        tools=tools,
        tool_choice="auto",
        messages=[
            {"role": "system", "content": "Você é um agente de execução que só age com base nos procedimentos."},
            {"role": "user", "content": pergunta},
            {"role": "system", "content": f"Base de conhecimento:\n{contexto}"}
        ]
    )
    return resposta

### Teste

In [25]:
pergunta = "Quero resetar a senha do gerente João, cpf 12345678901"
contexto = obter_contexto_rag(pergunta)

resposta = chamar_agente(pergunta, contexto)

choice = resposta.choices[0]

if choice.finish_reason == "tool_calls":
    print("Decidiu executar:", choice.message.tool_calls[0].function.name)
    print("Parâmetros:", choice.message.tool_calls[0].function.arguments)
else:
    print("Resposta textual:", choice.message.content)

Resposta textual: A redefinição de senha para um gerente deve ser autorizada pelo RH. Você tem a autorização do RH para prosseguir com a redefinição da senha do gerente João?
